In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import sys
import time
import requests
from itertools import cycle
from datetime import datetime
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchWindowException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait

In [2]:
from datetime import timedelta, date, datetime
def date_transform(org_str):
    current_datetime = datetime.now()
    try:
        org_str = org_str.split(' ')

        if org_str[1] == '分鐘':
            int_min = int(org_str[0])
            ans = current_datetime - timedelta(minutes=int_min)
            return ans

        if org_str[1] == '小時':
            int_hr = int(org_str[0])
            ans = current_datetime - timedelta(hours=int_hr)
            return ans

        if org_str[1] == '周' or org_str[1] == '週':
            int_days = int(org_str[0])*7
            ans = current_datetime - timedelta(days=int_days)
            return ans  

        if org_str[1] == '個月':
            int_days = int(org_str[0])*30
            ans = current_datetime - timedelta(days=int_days)
            return ans  

        if org_str[1] == '年':
            int_days = int(org_str[0])*365
            ans = current_datetime - timedelta(days=int_days)   
            return ans 
    except:
        return 0

def loop_the_page():
    #expand the comment for 5 time.
    try:
        for i in range(5):
            #show_more = driver.find_elements(By.ID,"see_next_pfbid02MvGzNvETiWhuayVy94oUKC3BJ9BuWEcAP4VoynFbCf6s2EtESE8SYknzUbwT2DEl")[-1]
            show_more = driver.find_elements(By.CLASS_NAME,"async_elem")[-1]

            ActionChains(driver).move_to_element(show_more).click().perform()
            time.sleep(2)
    except:
        pass
    
    #expand comment replies
    #the more comment button also name class, so maybe need to set a limit for the loop
    a = 0
    while a < 100:
        try:
            show_more_comment_replies = driver.find_element(By.CLASS_NAME,"async_elem")
            ActionChains(driver).move_to_element(show_more_comment_replies).click().perform()
            time.sleep(0.3)
            a=a+1
            #print(a)
        except:
            break
            


In [3]:
df_post = pd.read_excel('Facebook_Post_Level.xlsx', index_col=None)
df_post.head(1)

mobile_link = df_post["Mobile Link"].values.tolist()
post_uid = df_post["Post_UID"].values.tolist()
ref_bank = df_post["Reference Bank"].values.tolist()

#create dataframe to store it
df_comment = pd.DataFrame(columns=["Comment_UID",
                                   "Reference Bank",
                                   "Post Author",
                                   "Original Date",
                                   "New Datetime",
                                   "Content",  
                                   "Number of Likes"])

#store reaction link:
reaction_links = []

In [4]:
#code by pythonjar, not me
chrome_options = webdriver.ChromeOptions()
prefs = {"profile.default_content_setting_values.notifications" : 2}
chrome_options.add_experimental_option("prefs",prefs)

#specify the path to chromedriver.exe (download and save on your computer)
driver = webdriver.Chrome('chromedriver', chrome_options=chrome_options)

#open the webpage
driver.get("http://www.facebook.com")

#target username
username = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='email']")))
password = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='pass']")))

#enter username and password
username.clear()
username.send_keys("")
password.clear()
password.send_keys("")

#target the login button and click it
button = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))).click()

#wait for x seconds stop
time.sleep(6)


C:\Users\User\AppData\Local\Temp/ipykernel_28580/2729525120.py:7: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome('chromedriver', chrome_options=chrome_options)


In [5]:
for idx , link in enumerate(mobile_link):
    #get to target link
    driver.get(link)

    #buffer time to load the page
    time.sleep(2)
    html = driver.page_source
    soup = BeautifulSoup(html)
    
    #get reaction link
    likes_link = soup.find("a",{"class":"_45m8"}, href=True).get("href")
    reaction_links.append("https://m.facebook.com/"+ str(likes_link))

    #loop the page and expand founded element for 100 time
    loop_the_page()
    
    #pause for 2 sec to buffer
    time.sleep(2)
    #find the comment blocks
    html = driver.page_source
    soup = BeautifulSoup(html)
    for idx2, comment_per_block in enumerate(soup.find_all("div",{"class":"_2b04"})):
        author = comment_per_block.find("div",{"class":"_2b05"}).text
        
        date = comment_per_block.find("abbr",{"class":"_4ghv _2b0a"}).text
        new_date = date_transform(date)

        try:
            total_likes = comment_per_block.find("a",{"class":"_14v8 _4edm"}).text
        except:
            total_likes = 0

        comment_content = comment_per_block.find("div",{"class":"_2b06"}).text
        comment_content = comment_content[len(author):]

        #uid
        comment_uid_by_post=(post_uid[idx] + '-' + str(idx2).zfill(8))
        

        print(comment_uid_by_post)
        print(ref_bank[idx])
        print(author)
        print(comment_content)
        print(date)
        print(new_date)
        print(total_likes)
        print("="*20)


        df_comment.loc[len(df_comment)]= [comment_uid_by_post,
                                    ref_bank, 
                                    author, 
                                    date,
                                    new_date,
                                    comment_content,
                                    total_likes]
    if idx == 3:
        break

FB-00000000-00000000
Standard Chartered Hong Kong
Molly Eifie Tam 
我訓緊覺時都比人碌左16次 係咪送反里數做補償🤩🤩🤩
2 週
2022-12-07 13:18:30.111823
10
FB-00000000-00000001
Standard Chartered Hong Kong
Ming Kwan Kevin 
不如開group, 商討對策。 你點睇？
2 週
2022-12-07 13:18:30.113823
4
FB-00000000-00000002
Standard Chartered Hong Kong
Molly Eifie Tam 
Ming Kwan Kevin 其實佢點覆都係硬食
2 週
2022-12-07 13:18:30.116824
0
FB-00000000-00000003
Standard Chartered Hong Kong
Ming Kwan Kevin 
Molly Eifie Tam 人多，作用，效力大
2 週
2022-12-07 13:18:30.119824
0
FB-00000000-00000004
Standard Chartered Hong Kong
Kiki Leung 
Ming Kwan Kevin 好啊
2 週
2022-12-07 13:18:30.122826
0
FB-00000000-00000005
Standard Chartered Hong Kong
Jts Leung 
Ming Kwan Kevin 贊成開group
2 週
2022-12-07 13:18:30.125826
0
FB-00000000-00000006
Standard Chartered Hong Kong
Kenny Kin 
今朝兩部電話放係到打左3個鐘, 終於有人聽, 個transaction pending 都cancel 唔倒, 叫佢處理就話要退錢就當dispute transaction follow up, 真係退唔退倒要睇下期單...希望OK
2 週
2022-12-07 13:18:30.128826
0
FB-00000000-00000007
Standard Chartered Hong Kong
Chaca

In [6]:
'''
likes_link = soup.find("a",{"class":"_45m8"}, href=True).get("href")
likes_link = "https://m.facebook.com/"+ str(likes_link)
print(likes_link)
'''

'\nlikes_link = soup.find("a",{"class":"_45m8"}, href=True).get("href")\nlikes_link = "https://m.facebook.com/"+ str(likes_link)\nprint(likes_link)\n'

In [7]:
df_comment

,Comment_UID,Reference Bank,Post Author,Original Date,New Datetime,Content,Number of Likes
0,FB-00000000-00000000,"[Standard Chartered Hong Kong, Standard Charte...",Molly Eifie Tam,2 週,2022-12-07 13:18:30.111823,我訓緊覺時都比人碌左16次 係咪送反里數做補償🤩🤩🤩,10
1,FB-00000000-00000001,"[Standard Chartered Hong Kong, Standard Charte...",Ming Kwan Kevin,2 週,2022-12-07 13:18:30.113823,"不如開group, 商討對策。 你點睇？",4
2,FB-00000000-00000002,"[Standard Chartered Hong Kong, Standard Charte...",Molly Eifie Tam,2 週,2022-12-07 13:18:30.116824,Ming Kwan Kevin 其實佢點覆都係硬食,0
3,FB-00000000-00000003,"[Standard Chartered Hong Kong, Standard Charte...",Ming Kwan Kevin,2 週,2022-12-07 13:18:30.119824,Molly Eifie Tam 人多，作用，效力大,0
4,FB-00000000-00000004,"[Standard Chartered Hong Kong, Standard Charte...",Kiki Leung,2 週,2022-12-07 13:18:30.122826,Ming Kwan Kevin 好啊,0
...,...,...,...,...,...,...,...
559,FB-00000003-00000002,"[Standard Chartered Hong Kong, Standard Charte...",作者Standard Chartered Hong Kong,1 週,2022-12-14 13:22:15.564630,你好，Franky。由於現時熱線線路繁忙，等候時間比較長，請你耐心等候。或者可以先登入SC ...,0
560,FB-00000003-00000003,"[Standard Chartered Hong Kong, Standard Charte...",Ulf Waschbusch,2 週,2022-12-07 13:22:15.566630,I returned my credit card to your branch in Ce...,0
561,FB-00000003-00000004,"[Standard Chartered Hong Kong, Standard Charte...",Victor Chan,2 小時,2022-12-21 11:22:15.567631,渣打服務勁廢 有咩事熱線永遠無人聽,0
562,FB-00000003-00000005,"[Standard Chartered Hong Kong, Standard Charte...",Bewater Ho,1 週,2022-12-14 13:22:15.569631,請各位留心慎用渣打銀行，渣打安排下周一退款，表面上看似乎解決了問題，但實際上埋下更深的地雷。...,0


In [8]:
#output data
df_comment.to_excel('Facebook_Comment_Level.xlsx',encoding= 'utf-8-sig',index = None) 

C:\Users\User\anaconda3\lib\site-packages\pandas\util\_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)


In [9]:
'''
def fetch_likes():
    #click and pop
    view = driver.find_element(By.XPATH, '//*[@id="mount_0_0_fV"]/div/div[1]/div/div[5]/div/div/div[3]/div/div/div[1]/div[1]/div[4]/div[2]/div/div[2]/div/div/div/div[1]/div[2]/div/div/div/div/div/div/div/div/div/div[2]/div/div/div[4]/div/div/div[1]/div/div[1]/div/div[1]/div/span/div')
    view.click()
    likes_number_html = driver.page_source
    likes_number_soup = BeautifulSoup(likes_number_html)
    
    time.sleep(0.5)
    like_b = likes_number_soup.find_all("div",{"class":"x1i10hfl x6umtig x1b1mbwd xaqea5y xav7gou xe8uvvx xggy1nq x1o1ewxj x3x9cwd x1e5q0jg x13rtm0m x87ps6o x1lku1pv x1a2a7pz xjyslct xjbqb8w x18o3ruo x13fuv20 xu3j5b3 x1q0q8m5 x26u7qi x972fbf xcfux6l x1qhh985 xm0m39n x9f619 x1heor9g x1ypdohk xdj266r x11i5rnm xat24cr x1mh8g0r xexx8yu x4uap5 x18d9i69 xkhd6sd x1n2onr6 x16tdsg8 x1hl2dhg x1vjfegm x3nfvp2 xrbpyxo xng8ra x16dsc37"})
    for i in like_b:
        temp = i['aria-label'].split(",")
        if temp[0] == "全部":
            total = temp[1].replace(" ","")

        if temp[0] == "讚":
            like =  temp[1].replace(" ","")

        if temp[0] == "怒":
            angry = temp[1].replace(" ","")

        if temp[0] == "大心":
            heart =  temp[1].replace(" ","")

        if temp[0] == "哈":
            haha =  temp[1].replace(" ","")

        if temp[0] == "加油":
            add_oil =  temp[1].replace(" ","")

        if temp[0] == "嗚":
            cry =  temp[1].replace(" ","")

        if temp[0] == "哇":
            wow =  temp[1].replace(" ","")
            
    #close view
    close_view = driver.find_element(By.XPATH, '//*[@id="mount_0_0_fV"]/div/div[1]/div/div[6]/div/div/div[1]/div/div[2]/div/div/div/div/div/div/div[1]/div/div[2]/div')
    close_view.click()
            
    return total, like, angry, heart, haha, add_oil, cry, wow
'''

'\ndef fetch_likes():\n    #click and pop\n    view = driver.find_element(By.XPATH, \'//*[@id="mount_0_0_fV"]/div/div[1]/div/div[5]/div/div/div[3]/div/div/div[1]/div[1]/div[4]/div[2]/div/div[2]/div/div/div/div[1]/div[2]/div/div/div/div/div/div/div/div/div/div[2]/div/div/div[4]/div/div/div[1]/div/div[1]/div/div[1]/div/span/div\')\n    view.click()\n    likes_number_html = driver.page_source\n    likes_number_soup = BeautifulSoup(likes_number_html)\n    \n    time.sleep(0.5)\n    like_b = likes_number_soup.find_all("div",{"class":"x1i10hfl x6umtig x1b1mbwd xaqea5y xav7gou xe8uvvx xggy1nq x1o1ewxj x3x9cwd x1e5q0jg x13rtm0m x87ps6o x1lku1pv x1a2a7pz xjyslct xjbqb8w x18o3ruo x13fuv20 xu3j5b3 x1q0q8m5 x26u7qi x972fbf xcfux6l x1qhh985 xm0m39n x9f619 x1heor9g x1ypdohk xdj266r x11i5rnm xat24cr x1mh8g0r xexx8yu x4uap5 x18d9i69 xkhd6sd x1n2onr6 x16tdsg8 x1hl2dhg x1vjfegm x3nfvp2 xrbpyxo xng8ra x16dsc37"})\n    for i in like_b:\n        temp = i[\'aria-label\'].split(",")\n        if temp[0] == "全